## 커피지수 분석

각 지역별 커피지수를 구하여 시각화 한다.
- Starbucks, Coffee Bean, Ediya, 빽다방 커피 매장수를 파악하여 커피지수를 계산한다.
- 커피지수 = (Starbucks + Coffee Bean + Ediya) / 빽다방

### 1. 데이터 크롤링

In [181]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

- Starbucks Crawling

In [262]:
data = {
    'ins_lat':'', # 지정한 위도와 경도에서 가까운 순으로 나열
    'ins_lng':'',
    'p_sido_cd':'', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000000',
}
    
url = 'https://www.istarbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data)

r.text[:1000] # 수신된 데이터의 앞부분만 확인

'{"list":[{"seq":0,"sido_cd":null,"sido_nm":null,"gugun_cd":null,"gugun_nm":null,"code_order":null,"view_yn":null,"store_num":null,"sido":null,"gugun":null,"address":null,"new_img_nm":null,"p_pro_seq":0,"p_view_yn":null,"p_sido_cd":"","p_gugun_cd":"","p_store_nm":null,"p_theme_cd":null,"p_wireless_yn":null,"p_smoking_yn":null,"p_book_yn":null,"p_music_yn":null,"p_terrace_yn":null,"p_table_yn":null,"p_takeout_yn":null,"p_parking_yn":null,"p_dollar_assent":null,"p_card_recharge":null,"p_subway_yn":null,"stb_store_file_renew":null,"stb_store_theme_renew":null,"stb_store_time_renew":null,"stb_store_lsm":null,"s_code":"483","s_name":"광복중앙로","tel":"051-243-8473","fax":"051-243-8474","sido_code":"05","sido_name":"부산","gugun_code":"0510","gugun_name":"중구","addr":"부산광역시 중구 대청동2가 34-1","park_info":null,"new_state":"Y","theme_state":"T04@T05@T08@T16@T17@T20@T31@P80","new_bool":0,"search_text":"","ins_lat":"","ins_lng":"","in_distance":0,"out_distance":null,"all_search_cnt":-1,"addr_search_cnt":-1

In [263]:
jo = json.loads(r.text)
star_df = json_normalize(jo, 'list')

- 행(row)수, 서울 457개 매장

In [264]:
len(star_df)

1307

In [265]:
# 컬럼수 111개
star_df.columns

Index(['addr', 'addr_search_cnt', 'address', 'all_search_cnt', 'code_order',
       'cold_blew', 'defaultimage', 'disp', 'doro_address', 'espresso',
       ...
       't10', 't12', 't20', 't21', 't22', 'tel', 'theme_state', 'vSal',
       'view_yn', 'whcroad_yn'],
      dtype='object', length=119)

- 필요한 컬럼만 선택

In [266]:
# df = df[['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel']]
star_df = star_df[['s_name', 'sido_name', 'gugun_name']]
star_df.head()

,s_name,sido_name,gugun_name
0,광복중앙로,부산,중구
1,반야월이마트,대구,동구
2,남포동역,부산,중구
3,동탄이마트,경기,화성시
4,사상역,부산,사상구


In [268]:
star_df2 = star_df.copy()
star_df2 = star_df2.groupby([star_df2.sido_name, star_df2.gugun_name])
star_df2 = star_df2.count()
star_df2.columns = ['starbucks']
star_df2

starbucks
sido_name gugun_name           
강원        강릉시                 3
          동해시                 1
          속초시                 3
          춘천시                 5
          홍천군                 3
강원도       고성군                 1
          원주시                 6
경기        고양시 덕양구            10
          고양시 일산동구           13
          고양시 일산서구            9
          과천시                 3
          광명시                 8
          광주시                 2
          구리시                 3
          군포시                 4
          김포시                 8
          남양주시                7
          동두천시                2
          부천시 소사구             1
          부천시 오정구             6
          부천시 원미구             6
          성남시 분당구            36
          성남시 수정구             2
          성남시 중원구             2
          수원시 권선구             5
          수원시 영통구            11
          수원시 장안구             3
          수원시 팔달구             7
          시흥시                 5
          안산시 단원구             7
...                         ...
인천        서구                  6
          연수구                10
          중구                  7
전남        광양시                 1
          나주시                 1
          목포시                 5
          무안군                 1
          순천시                 4
          여수시                 5
전북        군산시                 2
          익산시                 3
          전주시 덕진구             4
          전주시 완산구             9
          정읍시                 1
제주        서귀포시                9
          제주시                 8
충남        계룡시                 1
          공주시                 2
          논산시                 1
          당진시                 2
          서산시                 2
          아산시                 4
          천안시 동남구             5
          천안시 서북구            10
충북        제천시                 1
          청주시 상당구             3
          청주시 서원구             3
          청주시 청원구             3
          청주시 흥덕구             6
          충주시                 4

[161 rows x 1 columns]

- Coffee Bean Crawling

In [275]:
sido = ['서울', '경기', '인천', '강원', '경남', '경북', '광주', '대구', '대전', '부산', '울산', '충남', '충북']

raw = ''
for v in sido:
    url = 'https://www.coffeebeankorea.com/store/store_data2.asp?lat=&lng=&chk1=0&chk2=0&chk3=0&chk4=0&chk5=0&chk6=0&chk7=0&chk8=0&chk9=0&keyword=&StoreLocal=%s' % v
    r = requests.post(url)
    
    raw += r.text

In [276]:
raw2 = raw.split()
raw2

['[{',
 'StoreNo',
 ':',
 "'341',",
 'StoreName',
 ':',
 "'서초동점',",
 'StoreTel',
 ':',
 "'02-3665-9507',",
 'StoreAddress',
 ':',
 "'서울특별시",
 '서초구',
 "반포대로70'",
 ',',
 'StoreOpendate',
 ':',
 "'월",
 '-',
 '금',
 '07:30~22:00',
 '|',
 '주말/공휴일',
 "08:00~22:00'",
 ',',
 'StoreLAT',
 ':',
 "'37.48669594836945',",
 'StoreLNG',
 ':',
 "'127.01053356053066',",
 'StoreShortDesc',
 ':',
 "'',",
 'StoreEmail',
 ':',
 "'cbk431@coffeebeankorea.com'",
 ',',
 'StoreDecaffeine',
 ':',
 "'1'",
 ',',
 'StoreSoymilk',
 ':',
 "'1',",
 'StoreWifi',
 ':',
 "'1',",
 'StoreParking',
 ':',
 "'0',",
 'Storesmoking',
 ':',
 "'1'",
 ',',
 'StoreMachine',
 ':',
 "'0'",
 ',',
 'StoreCapsule',
 ':',
 "'1',",
 'StoreCake',
 ':',
 "'1',",
 'StoreDrive',
 ':',
 "'0',",
 'Distant',
 ':',
 "'',",
 'sort',
 ':',
 "'1'",
 ',',
 'StoreImage1',
 ':',
 "'1_80.jpg',",
 'StoreImage2',
 ':',
 "'신규매장_1.jpg',",
 'StoreImage3',
 ':',
 "'3_75.jpg',",
 'StoreImage4',
 ':',
 "'4_67.jpg',",
 'StoreImage5',
 ':',
 "'5_47.jpg'",
 ',',
 '

In [277]:
coffee_bean = []
for i in range(len(raw2)-3):
    if raw2[i] == 'StoreAddress':
        coffee_bean.append([raw2[i+2], raw2[i+3]])

In [291]:
coffee_bean[3] = ["'서울특별시", "송파구"]

"'서울"

In [292]:
for i in range(len(coffee_bean)):
    coffee_bean[i][0] = coffee_bean[i][0].replace("'", '')

In [305]:
cb_df = pd.DataFrame(coffee_bean)
cb_df.columns = ['sido_name', 'gugun_name']
cb_df['coffe_bean'] = 1
cb_df = cb_df.groupby(['sido_name', 'gugun_name'])
cb_df = cb_df.count()
cb_df

coffe_bean
sido_name gugun_name            
          인천                   1
강원도       원주시                  1
경기        김포시                  1
          수원시                  2
          용인시                  1
          이천시                  1
          화성시                  1
경기도       고양시                  7
          광명시                  1
          구리시                  1
          군포시                  1
          김포시                  1
          남양주시                 1
          부천시                  4
          성남시                 10
          수원시                  6
          안양시                  4
          용인시                  1
          파주시                  2
          평택시                  1
          하남시                  1
          화성시                  4
경남        창원시                  2
경상남도      진주시                  1
경상북도      경주시                  1
          포항시                  1
광주        서구                   1
대구광역시     중구                   1
대전광역시     서구                   1
          유성구                  1
...                          ...
서울시       성북구                  1
          송파구                  4
          양천구                  1
          영등포구                 2
          용산구                  2
          은평구                  1
          종로구                  2
          중구                   3
서울특별시     강남구                  8
          강동구                  1
          강북구                  1
          강서구                  2
          광진구                  1
          구로구                  1
          마포구                  2
          서초구                  3
          성북구                  1
          송파구                  5
          영등포구                 1
          종로구                  3
          중랑구                  1
세종특별자치시   세종로                  1
울산광역시     남구                   2
인천광역시     남동구                  1
          부평구                  1
          연수구                  1
          중구                   3
인천시       중구                   1
충남        천안시                  1
충북        청주시                  2

[80 rows x 1 columns]